In [1]:
# import fitz  # PyMuPDF
from PyPDF2 import PdfReader
import os
from nltk.tokenize import sent_tokenize

# def extract_text_from_pdf(pdf_path):
#     """Extracts text from a PDF file."""
#     doc = fitz.open(pdf_path)
#     text = ""
#     for page_num in range(len(doc)):
#         page = doc.load_page(page_num)
#         text += page.get_text()
#     return text

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file with pypdf2."""
    reader = PdfReader(pdf_path)
    text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        text += page.extract_text()
    return text
# Example usage:
pdf_folder = 'data/'
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith('.pdf')]

# create a dictionary to store the text of each pdf and the metadata and if there is a txt with the same name
contex = {}
for pdf_file in pdf_files:
    contex[pdf_file]={}
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    contex[pdf_file]['pdf'] = text
    # get the metadata
    #metadata = fitz.open(pdf_path).metadata
    metadata = PdfReader(pdf_path).metadata
    contex[pdf_file]['metadata'] = metadata
    # check if there is a txt with the same name
    txt_path = os.path.join(pdf_folder, pdf_file.replace('.pdf', '.txt'))
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            contex[pdf_file]['transcript'] = file.read()

    else:
        contex[pdf_file]['transcript'] = None

In [30]:
# one file:
pdf_folder = 'data/batch/'
metting_name = "gib_mcp_rgc_min__2024-04-09__01"
# read the info from a xlsx file
import pandas as pd
df = pd.read_excel("data/meetingmap.xlsx")
df = df.set_index('standard name')
# get the info of the meeting
meeting_info = df.loc[metting_name+".pdf"]
# return all the columns for the meeting
meeting_info = meeting_info.to_dict()
meeting_info


{'location': 'gibsons',
 'location type': 'municipality',
 'meeting type': 'regular_council',
 'data type': 'minutes',
 'meeting date': Timestamp('2024-04-09 00:00:00'),
 'transcript': 'Yes',
 'comment': nan}

In [32]:
import fitz  # PyMuPDF
import os
from nltk.tokenize import sent_tokenize

def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

# Example usage:
pdf_files = [metting_name+".pdf"]

# create a dictionary to store the text of each pdf and the metadata and if there is a txt with the same name
contex = {}
for pdf_file in pdf_files:
    contex[pdf_file]={}
    pdf_path = os.path.join(pdf_folder, pdf_file)
    text = extract_text_from_pdf(pdf_path)
    contex[pdf_file]['pdf'] = text
    # get the metadata
    metadata = fitz.open(pdf_path).metadata
    contex[pdf_file]['metadata'] = metadata
    # check if there is a txt with the same name
    txt_path = os.path.join(pdf_folder, pdf_file.replace('.pdf', '.txt'))
    if os.path.exists(txt_path):
        with open(txt_path, 'r') as file:
            contex[pdf_file]['transcript'] = file.read()

    else:
        contex[pdf_file]['transcript'] = None

In [40]:
contex[pdf_files[0]].keys()

dict_keys(['pdf', 'metadata', 'transcript'])

In [43]:
def chunk_text(text, chunk_size=512, overlap=256):
    """Divides text into overlapping chunks."""
    sentences = sent_tokenize(text)
    chunks = []
    chunk = []

    current_length = 0
    for sentence in sentences:
        chunk.append(sentence)
        current_length += len(sentence.split())

        if current_length >= chunk_size:
            chunks.append(" ".join(chunk))
            chunk = chunk[-(overlap // len(sentence.split())):]  # Start next chunk with the overlap
            current_length = len(" ".join(chunk).split())

    if chunk:
        chunks.append(" ".join(chunk))

    return chunks

# Example usage:
input_text = [contex[pdf_file]['pdf'] for pdf_file in pdf_files]
for text in input_text:
    chunks = chunk_text(text)

In [79]:
# better option
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1024, chunk_overlap=50
)
input_txt = [contex[pdf_file]['pdf'] for pdf_file in pdf_files]
for text in input_text:
    all_splits = text_splitter.split_text(text)


In [80]:
len(all_splits)


2

In [ ]:
# from langchain_community.embeddings import GPT4AllEmbeddings
# from langchain_community.vectorstores import Chroma
# model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
# gpt4all_kwargs = {'allow_download': 'True'}

# embedding = GPT4AllEmbeddings(  model_name=model_name,
#     gpt4all_kwargs=gpt4all_kwargs)
# # Index
# vectorstore = Chroma.from_documents(
#     documents=all_splits,
#     collection_name="rag-chroma",
#     embedding=embedding,
# )
# retriever = vectorstore.as_retriever()

In [47]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
import json

In [122]:

model = Ollama(model="llama3")

PROMPT ="""<|system|>
You are an expert in finding proposals in a meeting note. You are provided with a context delimited by ### which is a chunk of a large meeting note. The goal is to find unique individual proposals in the meeting. Your task is to find the count of new proposal and write the title of the proposal(s). Seperate different proposals by  ||. You also will be given the title from the previous proposal, Do NOT count it as a new one if it was in the previous chunk. If a proposal is not complete (does not have the result), do not count it. Do NOT write more than 10 words for each proposal.  Your output style should be this: "<num proposal> || <proposal title> || <proposal title> ...". Here are two examples:
example 1 
Context:###Minutes of the Council meeting of February 6, 2024, be approved.\n CARRIED UNANIMOUSLY \n Council Meeting\n Minutes, February 27, 2024 3\n 3. Council (City Finance and Services) \n MOVED by Councillor Dominato\n SECONDED by Councillor Carr\n THAT the Minutes of the Council meeting following the Standing Committee on City\n Finance and Services meeting of February 7, 2024, be approved.\n CARRIED UNANIMOUSLY\n 4. Court of Revision (Business Improvement Areas) - February 8, 2024\n MOVED by Councillor Bligh\n SECONDED by Councillor Zhou\n  THAT the Minutes of the Court of Revision (Business Improvement Areas) meeting of\n February 8, 2024, be approved.\n CARRIED UNANIMOUSLY\n MATTERS ADOPTED ON CONSENT\n MOVED by Councillor Carr ###
ast proposal: ###  the Minutes of the Council meeting of February 6, 2024, be approved. ###
assistant: 2 || the Minutes of the Council meeting following the Standing Committee on City Finance and Services, be approved || the Minutes of the Court of Revision (Business Improvement Areas) meeting of February 8, 2024, be approved.

example 2 
Context: ###  Tax Rates Bylaw”:\n THAT “Tax Rates Bylaw, 2024, No. 9017” be considered.\n CARRIED UNANIMOUSLY \n R2024-04-22/10\n BYLAW FIRST, SECOND AND THIRD READINGS \n 11. “Tax Rates Bylaw, 2024, No. 9017” \n Moved by Councillor Valente, seconded by Councillor Shahriari\n THAT “Tax Rates Bylaw, 2024, No. 9017” be given first and second readings;\n AND THAT “Tax Rates Bylaw, 2024, No. 9017” be given third reading.\n CARRIED UNANIMOUSLY\n R2024-04-22/11\n PUBLIC CLARIFICATION PERIOD\n Mayor Buchanan declared a recess at 9:44 pm for the Public Clarification Period and  \n Report: Manager, Public Realm Infrastructure, April 10, 2024\n Moved by Councillor Valente, seconded by Councillor Girard AND THAT staff be directed of the\n Lonsdale Highway Overpass Mobility Improvements with the Phase 1 concept of the\n Upper Levels Greenway to develop a coordinated  \n Business Licensing ###
last proposal: ### Tax Rates Bylaw, 2024, No. 9017” be considered.###
assistant: 1 || Tax Rates Bylaw, 2024, No. 9017” be given first and second readings
<|end|>
<|user|>
Context:
###
{context}
###

Last proposal:
###
{latest_proposal}
###
<|end|>
<|assistant|> """ 

PROMPT ="""<|system|>
You are an expert in finding proposals in a meeting note. You are provided with a context delimited by ### which is a chunk of a large meeting note. The goal is to find unique individual proposals in the meeting. Your task is to find the count of unique proposal and write the title of the proposal(s). Seperate different proposals by  ||. If a proposal is not complete (does not have the result), do not count it. DO NOT write less than 5 or more than 15 words for each proposal.  

Your output style should be this: "|| <num proposal, n> || <proposal 1 title> || <proposal 2 title> ... || <proposal n title>". 

Let's think step by step. Here are the steps to solve the task:
1. Find the pattern in the chunk that indicates a proposal.
2. Find the number of unique proposals in the chunk. 
3. Make sure all content about each proposal including the title, proposer, seconder, and the result are included and they are unique.
4. Write a title (min 5 to max 15 words) for those proposals delimited by ||.

Here is an example: 
Context: ###Minutes of the Council meeting of February 6, 2024, be approved.\n CARRIED UNANIMOUSLY \n Council Meeting\n Minutes, February 27, 2024 3\n 3. Council (City Finance and Services) \n MOVED by Councillor Dominato\n SECONDED by Councillor Carr\n THAT the Minutes of the Council meeting following the Standing Committee on City\n Finance and Services meeting of February 7, 2024, be approved.\n CARRIED UNANIMOUSLY\n 4. Court of Revision (Business Improvement Areas) - February 8, 2024\n MOVED by Councillor Bligh\n SECONDED by Councillor Zhou\n  THAT the Minutes of the Court of Revision (Business Improvement Areas) meeting of\n February 8, 2024, be approved.\n CARRIED UNANIMOUSLY\n MATTERS ADOPTED ON CONSENT\n MOVED by Councillor Carr \n Tax Rates Bylaw”:\n THAT “Tax Rates Bylaw, 2024, No. 9017” be considered.\n R2024-04-22/10\n BYLAW FIRST,   ###
thinking steps:
1. It is constantly using the word THAT to indicate a proposal.
2. found 4 proposals which has THAT
3. the last proposal does not have the result, so it is not a complete proposal.
4. I have 3 proposals, and will write short title for them.

assistant: || 3 ||  the Minutes of the Council meeting of February 6, 2024, be approved. || the Minutes of the Council meeting following the Standing Committee on City Finance and Services, be approved || the Minutes of the Court of Revision (Business Improvement Areas) meeting be approved.
<|end|>

<|user|>
Context:
###
{context}
###
<|end|>

<|assistant|> """ 
prompt = ChatPromptTemplate.from_template(PROMPT)

chain = prompt | model
proposal_dictionary = {}
proposal_count_dictionary = {}
total_chunks = len(all_splits)
iterator = 1
while iterator <= total_chunks:
    context = all_splits[iterator-1]
    print(f'Processing chunk {iterator} of {total_chunks} ...')
    latest_proposal = chain.invoke({'context': context})
    # parse the latest_proposal and seperate them if it has special token <sep>
    latest_proposal = latest_proposal.split('||') 
    if len(latest_proposal) == 0:
        print(f"There was an error in chunk {iterator}, no || found, running again")
        continue
    try:
        int(latest_proposal[1])
    except:
        print(f"There was an error in chunk {iterator}, running again")
        print(latest_proposal[1])
        continue
    print(f"{latest_proposal[1]} new proposals was found in this chunk")
    proposal_dictionary[iterator-1] = latest_proposal[2:]
    # checking for their lenght 
    for i in reversed(range(len(proposal_dictionary[iterator-1]))):
        if len(proposal_dictionary[iterator-1][i]) < 20:
            # remove this proposal
            print(f" Removing proposal (lenght limit) '{proposal_dictionary[iterator-1][i]}'")
            proposal_dictionary[iterator-1].pop(i)
    proposal_count_dictionary[iterator-1] = int(latest_proposal[1])
    if len(proposal_dictionary[iterator-1]) != proposal_count_dictionary[iterator-1]:
        print(f"Warning, proposals in chunk {iterator} does not match the count")
    iterator += 1


Processing chunk 1 of 2 ...
 5  new proposals was found in this chunk
 Removing proposal (lenght limit) ' '
Processing chunk 2 of 2 ...
 3  new proposals was found in this chunk
 Removing proposal (lenght limit) '
<|end|>'


In [123]:
# convert proposal_dictionary to all_proposals list
all_proposals = []
chunks =[]
for  key, value in proposal_dictionary.items():
    all_proposals.extend(value)
    # populate chunk with the keys on all elements of the value
    chunks.extend([key for _ in range(len(value)) ])
print(len(all_proposals))
print(chunks)

8
[0, 0, 0, 0, 0, 1, 1, 1]


In [124]:
# find the count for each proposal in the chunks
# count the number of elements in the chunks
proposal_count_returned = {i:chunks.count(i) for i in set(chunks)}
# find the chunks that the two dictionaries are different
diff = {k: proposal_count_returned[k] - proposal_count_dictionary[k] for k in proposal_count_returned if k in proposal_count_dictionary and proposal_count_returned[k] != proposal_count_dictionary[k]}

diff

{}

In [125]:
# if the is differece, we need to look at the proposals that are different
from sklearn.feature_extraction.text import TfidfVectorizer

def get_similarity_score(proposals):
    pairwise_similarity = {}
    vect = TfidfVectorizer(min_df=1, stop_words="english")
    # select two adjacent proposals and calculate the cosine similarity
    for i in range(len(proposals)-1):
        tfidf = vect.fit_transform([proposals[i], proposals[i+1]])                                                                                                                                                                                                                       
        pairwise_similarity[f'{i}-{i+1}']=(tfidf * tfidf.T).toarray()[0][1]
    return pairwise_similarity

In [126]:
for k in diff.keys():
    print(f"Checking the proposals in chunk {k}")
    print(get_similarity_score(proposal_dictionary[k]))


In [27]:
# save the proposal dictionary to a txt file
with open('data/proposals_{}.txt'.format(metting_name), 'w') as file:
    json.dump(proposal_dictionary, file)

In [18]:
# load the proposal dictionary from a txt file
with open('data/proposals_{}.txt'.format(metting_name), 'r') as file:
    proposal_dictionary = json.load(file)

In [128]:
# we need to check if the proposals are unique
# we can use the cosine similarity to check if the proposals are unique

# get the cosine similarity between the proposals
from sklearn.feature_extraction.text import TfidfVectorizer


vect = TfidfVectorizer(min_df=1, stop_words="english")
to_remove = []
# select two adjacent proposals and calculate the cosine similarity
for i in range(len(all_proposals)-1):
    tfidf = vect.fit_transform([all_proposals[i], all_proposals[i+1]])                                                                                                                                                                                                                       
    pairwise_similarity = tfidf * tfidf.T 
    if pairwise_similarity.toarray()[0][1] > 0.5:
        to_remove.append(i+1)
        print(f"Proposal {i} and {i+1} are similar with score {pairwise_similarity.toarray()[0][1]}")
        print(all_proposals[i])
        print(all_proposals[i+1])  
        print('-------------------')                                                                                                                                                                                             


In [129]:
# remove the similar proposals
unique_proposals = [all_proposals[i] for i in range(len(all_proposals)) if i not in to_remove]
unique_chunks = [chunks[i] for i in range(len(chunks)) if i not in to_remove]

In [130]:
# convert back to dictionary
unique_proposal_dictionary = {}
for i in range(len(unique_chunks)):
    if unique_chunks[i] in unique_proposal_dictionary:
        unique_proposal_dictionary[unique_chunks[i]].append(unique_proposals[i])
    else:
        unique_proposal_dictionary[unique_chunks[i]] = [unique_proposals[i]]

In [131]:
unique_proposal_dictionary

{0: [' the minutes of the Regular Council meeting held March 19, 2024 be adopted. ',
  ' the Development Permit Delegation Authority Amendment Bylaw No.1054-04, 2024 be adopted. ',
  ' the revised preliminary 5-year capital plan for general services be integrated into the 2024-2028 Financial Plan with the exception of the Dog Park and Pickleball projects which are to be removed from the capital plan and referred to the 2025 Parks Master Plan for discussion; AND THAT the proposed budget for the Healthy Harbours Project be approved, with the 2024 portion being funded by accumulated surplus. ',
  ' the 2024 annual tax rates be prepared authorizing an overall 8% tax increase (reflecting 3% for general operations and 5% for future policing costs). ',
  ' the 2024-2028 Financial Plan Bylaw and the 2024 Annual Tax Rate Bylaw be prepared for Council approval.'],
 1: [' Parcel Tax Roll Review Panel – Water/Sewer/Community Recreation be approved ',
  ' Canada BC Housing Benefit Program Memorandu

In [134]:
# get all the information for this specific meeting from the proposals_for_review.xlsx file (training only) sheet review
df = pd.read_excel("data/proposals_for_review.xlsx", sheet_name="review")
df = df.set_index('standard_name')
# get the list of proposals where the meeting name is the same as the current meeting
meeting_proposals = df.loc[metting_name+".pdf"]

meeting_proposals

,sectionid,subsectionid,policy_prop,theme,terms,policy_prop_future_date,future_action_date_type,vote_result,theme_as_reviewed,terms_to_remove,terms_to_add,comments,Wrong Cut,future_date_as_reviewed,sentence_suggesting_future_action,voting_result_as_reviewed,Validated?
standard_name,,,,,,,,,,,,,,,,,
gib_mcp_rgc_min__2024-04-09__01.pdf,4.0,1.0,R2024-66 2024-2028 Preliminary General Service...,unclassified,the Dog\nPark and Pickleball projects,NaN,NaN,CARRIED,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,Yes
gib_mcp_rgc_min__2024-04-09__01.pdf,5.0,1.0,R2024-69 Parcel Tax Roll Review Panel - Water/...,housing,community,2024-05-07,exact,CARRIED,drop,community,NaN,NaN,NaN,NaT,NaN,NaN,Yes
gib_mcp_rgc_min__2024-04-09__01.pdf,6.0,3.0,R2024-72 Allocation of Community Event Fund:\n...,housing,community,NaN,NaN,CARRIED,drop,community,NaN,NaN,NaN,NaT,NaN,NaN,Yes


In [160]:
df = pd.read_excel("data/proposals_for_review.xlsx", sheet_name="themes")
themes = df['theme'].tolist()
themes = ['Uncategorized', 'food', 'housing', 'children', 'climate', 'health', 'environment', 'indigenous', 'women', 'diversity', 'poverty', 'psychoactive', 'mental']


In [167]:
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_experimental.llms.ollama_functions import OllamaFunctions, convert_to_ollama_tool


# Schema for structured response
class Proposal(BaseModel):
    section: str = Field(description="The section and subsection of the proposal", required=True)
    title: str = Field(description="A short title of the proposal", required=True)
    policy_prop: str = Field(description="Full proposal content", required=True)
    theme: str = Field(description=f"categorical value from themes: {themes}", required=True)
    vote_result: str = Field(description="The result of the vote", required=True)
    future_date: str = Field(description="The future date of the proposal", required=False)


# Prompt template
prompt = PromptTemplate.from_template(
    """<|system|> You are an expert in extracting information related to a proposal from chunk of a text. Your task is to get the proposal from the user and find relevant information only from the given chunk of context delimited by ###. Your answer should follow the following scheme:
    section: str = Field(description="The section and subsection of the proposal", required=True)
    title: str = Field(description="A short title of the proposal", required=True)
    policy_prop: str = Field(description="Full proposal content", required=True)
    theme: str = Field(description="the most related categorical value from this  list: {themes}", required=True)
    vote_result: str = Field(description="The result of the vote", required=True)
    future_date: str = Field(description="The future date of the proposal", required=False)
    
Let's think step by step. Here are the steps to solve the task:
1. Find the given proposal from the user. It is delimted by *** and followed after the PROPOSAL word. 
2. Locate the given proposal in the context delimited by ###. Try to find where is the beginning and the end of that proposal.
3. Look for different required information of that proposal including the section, title, policy_prop, theme, vote_result, and future_date (only if exist).
3. Make sure all retrieved information are extracted from the context. If it was not, return all the sections with 'NOT FOUND'.
4. Call the tool_calls of langchain_experimental.llms.ollama_functions.OllamaFunctions with the model phi3
5. Make sure the return type are correct and in the correct format

Here is an example: 
Context: ###Minutes of the Council meeting of February 6, 2024, be approved.\n CARRIED UNANIMOUSLY \n Council Meeting\n Minutes, February 27, 2024 3\n 3. Council (City Finance and Services) \n MOVED by Councillor Dominato\n SECONDED by Councillor Carr\n THAT the Minutes of the Council meeting following the Standing Committee on City\n Finance and Services meeting of February 7, 2024, be approved.\n CARRIED UNANIMOUSLY\n 4. Court of Revision (Business Improvement Areas) - February 8, 2024\n MOVED by Councillor Bligh\n SECONDED by Councillor Zhou\n  THAT the Minutes of the Court of Revision (Business Improvement Areas) meeting of\n February 8, 2024, be approved.\n CARRIED UNANIMOUSLY\n MATTERS ADOPTED ON CONSENT\n MOVED by Councillor Carr \n Tax Rates Bylaw”:\n THAT “Tax Rates Bylaw, 2024, No. 9017” be considered.\n R2024-04-22/10\n BYLAW FIRST,   ###
PROPOSAL: ***the Minutes of the Council meeting following the Standing Committee on City Finance and Services, be approved ***

assistant: 
thinking steps:
1. Found the proposal that is delimited by *** followed by PROPOSAL
2. The begining of the proposal is 3. Council (City Finance and Services) and the end is February 7, 2024, be approved. CARRIED UNANIMOUSLY
3. section is 3, title is Approve of the Minutes of the Council meeting in Standing Committee on City Finance, policy prop is started after THAT until CARRIED UNANIMOUSLY, theme is Uncategorized, vote result is CARRIED UNANIMOUSLY, future date is February 7, 2024 
4. I will call the tool_calls of langchain_experimental.llms.ollama_functions.OllamaFunctions to make sure the structure is correct.
5. I double check that the results are valid and in the correct format.

tool_calls ...
<|end|>

<|user|>
Context:
###
{context}
###

PROPOSAL:
###
{proposal}
###
<|end|>

<|assistant|> """
)

# Chain
llm = OllamaFunctions(model="phi3",format="json", temperature=0)
structured_llm = llm.with_structured_output(Proposal)
chain = prompt | structured_llm

iterator = 0
context = all_splits[iterator-1]
proposal = unique_proposal_dictionary[iterator][0]

result = chain.invoke({'themes':themes ,'context': context, 'proposal': proposal})
result


 the minutes of the Regular Council meeting held March 19, 2024 be adopted. 
